In [14]:
from datetime import datetime
from time import time
from csv import DictReader
from math import exp, log, sqrt
from random import random
import pickle
import os

from ml_metrics_auc import auc # 추가(Diaman)

In [24]:
# A, paths
train='train_sample_df_2.csv'
test='test_sample_df_2.csv'
submission = 'ftrlsub.csv'  # path of to be outputted submission file

In [16]:
# B, model
alpha = .1  # learning rate
beta = 1.   # smoothing parameter for adaptive learning rate
L1 = 1.     # L1 regularization, larger value means more regularized
L2 = 0.     # L2 regularization, larger value means more regularized

In [17]:
# C, feature/hash trick
D = 2 ** 24             # number of weights to use
                        # 우리 데이터의 경우 : (OHE 만들어 놨다는 가정하에)변수의 개수
interaction = False     # whether to enable poly2 feature interactions

# D, training/validation
epoch = 1       # learn training data for N passes
holdafter = None   # data after date N (exclusive) are used as validation
holdout = None  # use every N training instance for holdout validation

show_auc = False # 추가(Diaman)

In [18]:
##############################################################################
# class, function, generator definitions #####################################
##############################################################################

class ftrl_proximal(object):
    ''' Our main algorithm: Follow the regularized leader - proximal

        In short,
        this is an adaptive-learning-rate sparse logistic-regression with
        efficient L1-L2-regularization

        Reference:
        http://www.eecs.tufts.edu/~dsculley/papers/ad-click-prediction.pdf
    '''

    def __init__(self, alpha, beta, L1, L2, D, interaction):
        # parameters
        self.alpha = alpha
        self.beta = beta
        self.L1 = L1
        self.L2 = L2

        # feature related parameters
        self.D = D
        self.interaction = interaction

        # model
        # n: squared sum of past gradients
        # z: weights
        # w: lazy weights
        self.n = [0.] * D
        
    
        self.z = [random() for k in range(D)]#[0.] * D
        self.w = {}

    def _indices(self, x):
        ''' A helper generator that yields the indices in x

            The purpose of this generator is to make the following
            code a bit cleaner when doing feature interaction.
        '''

        # first yield index of the bias term
        yield 0

        # then yield the normal indices
        for index in x:  # hash 처리 된 x값
            yield index
            

    def predict(self, x):
        ''' Get probability estimation on x

            INPUT:
                x: features

            OUTPUT:
                probability of p(y = 1 | x; w)
        '''

        # parameters
        alpha = self.alpha
        beta = self.beta
        L1 = self.L1
        L2 = self.L2

        # model
        n = self.n
        z = self.z
        w = {}

        # wTx is the inner product of w and x
        wTx = 0.
        for i in self._indices(x):  # 인덱스 존재하는 값만 w를 계산
                                    # 우리의 경우 nonzero x를 가져오면 됨
            sign = -1. if z[i] < 0 else 1.  # get sign of z[i]

            # build w on the fly using z and n, hence the name - lazy weights
            # we are doing this at prediction instead of update time is because
            # this allows us for not storing the complete w
            if sign * z[i] <= L1:
                # w[i] vanishes due to L1 regularization
                w[i] = 0.
            else:
                # apply prediction time L1, L2 regularization to z and get w
                w[i] = (sign * L1 - z[i]) / ((beta + sqrt(n[i])) / alpha + L2)

            wTx += w[i]

        # cache the current w for update stage
        self.w = w

        # bounded sigmoid function, this is the probability estimation
        return 1. / (1. + exp(-max(min(wTx, 35.), -35.)))

    def update(self, x, p, y):
        ''' Update model using x, p, y

            INPUT:
                x: feature, a list of indices
                p: click probability prediction of our model
                y: answer

            MODIFIES:
                self.n: increase by squared gradient
                self.z: weights
        '''

        # parameter
        alpha = self.alpha

        # model
        n = self.n
        z = self.z
        w = self.w

        # gradient under logloss
        g = p - y

        # update z and n
        for i in self._indices(x):
            sigma = (sqrt(n[i] + g * g) - sqrt(n[i])) / alpha
            z[i] += g - sigma * w[i]
            n[i] += g * g


def logloss(p, y):
    ''' FUNCTION: Bounded logloss

        INPUT:
            p: our prediction
            y: real answer

        OUTPUT:
            logarithmic loss of p given y
    '''

    p = max(min(p, 1. - 10e-15), 10e-15)
    return -log(p) if y == 1. else -log(1. - p)

In [19]:
def data(path, D):
    ''' GENERATOR: Apply hash-trick to the original csv row
                   and for simplicity, we one-hot-encode everything

        INPUT:
            path: path to training or testing file
            D: the max index that we can hash to

        YIELDS:
            ID: id of the instance, mainly useless
            x: a list of hashed and one-hot-encoded 'indices'
               we only need the index since all values are either 0 or 1
            y: y = 1 if we have a click, else we have y = 0
    '''

    for t, row in enumerate(DictReader(open(path), delimiter=',')):
        # process id
        #print row
        
        try:
            display_id = row['display_id']
            ad_id = row['ad_id']
            timestamp_event = row['timestamp_event']
            is_leak = row['is_leak']
            day_event = row['day_event']
            
            del row['display_id']
            del row['ad_id']
            del row['timestamp_event']
            del row['is_leak']
            del row['day_event']
            
        except:
            pass
        # process clicks
        y = 0.
        target='label'#'IsClick' 
        if target in row:
            if row[target] == '1':
                y = 1.
            del row[target]

        # extract date

        # turn hour really into hour, it was originally YYMMDDHH

        # build x
        x = []
        for key in row:
            value = row[key]

            # one-hot encode everything with hash trick
            index = abs(hash(key + '_' + value)) % D
            x.append(index)

        yield t, display_id, ad_id,  x, y

In [20]:
# ##############################################################################
# start training #############################################################
##############################################################################

start = time()

# initialize ourselves a learner
learner = ftrl_proximal(alpha, beta, L1, L2, D, interaction)

# start training
print('trainning models...')

for e in range(epoch):
    loss = 0.
    count = 0
    for t, display_id, ad_id, x, y in data(train, D):  # data is a generator

        p = learner.predict(x)
        loss += logloss(p, y)
        learner.update(x, p, y)
        count+=1
        if count%10000==0:
            #print count,loss/count
            print('%s\tencountered: %d\tcurrent logloss: %f' % (
                datetime.now(), count, loss/count))
            # x = count, y = loss/count로 plot 그리기
            
count=0
loss=0


print('training took %0.3fm' % ((time() - start) / 60))

trainning models...
2018-12-10 16:19:30.384387	encountered: 10000	current logloss: 0.467183
2018-12-10 16:19:31.020272	encountered: 20000	current logloss: 0.464576
2018-12-10 16:19:31.670658	encountered: 30000	current logloss: 0.437086
2018-12-10 16:19:32.325855	encountered: 40000	current logloss: 0.452525
2018-12-10 16:19:32.961398	encountered: 50000	current logloss: 0.458536
2018-12-10 16:19:33.598099	encountered: 60000	current logloss: 0.457684
2018-12-10 16:19:34.233378	encountered: 70000	current logloss: 0.450643
2018-12-10 16:19:34.871292	encountered: 80000	current logloss: 0.448569
2018-12-10 16:19:35.507794	encountered: 90000	current logloss: 0.448966
2018-12-10 16:19:36.147967	encountered: 100000	current logloss: 0.447056
2018-12-10 16:19:36.786914	encountered: 110000	current logloss: 0.450472
2018-12-10 16:19:37.429814	encountered: 120000	current logloss: 0.453173
2018-12-10 16:19:38.070055	encountered: 130000	current logloss: 0.449046
2018-12-10 16:19:38.712383	encountered: 

In [21]:
# auc 
def tied_rank(x):
    """
    Computes the tied rank of elements in x.
    This function computes the tied rank of elements in x.
    Parameters
    ----------
    x : list of numbers, numpy array
    Returns
    -------
    score : list of numbers
            The tied rank f each element in x
    """
    sorted_x = sorted(zip(x,range(len(x))))
    r = [0 for k in x]
    cur_val = sorted_x[0][0]
    last_rank = 0
    for i in range(len(sorted_x)):
        if cur_val != sorted_x[i][0]:
            cur_val = sorted_x[i][0]
            for j in range(last_rank, i): 
                r[sorted_x[j][1]] = float(last_rank+1+i)/2.0
            last_rank = i
        if i==len(sorted_x)-1:
            for j in range(last_rank, i+1): 
                r[sorted_x[j][1]] = float(last_rank+i+2)/2.0
    return r

def auc(y, p):
    """
    Computes the area under the receiver-operater characteristic (AUC)
    This function computes the AUC error metric for binary classification.
    Parameters
    ----------
    actual : list of binary numbers, numpy array
             The ground truth value
    posterior : same type as actual
                Defines a ranking on the binary numbers, from most likely to
                be positive to least likely to be positive.
    Returns
    -------
    score : double
            The mean squared error between actual and posterior
    """
    r = tied_rank(p)
    num_positive = len([0 for x in y if x==1])
    num_negative = len(y)-num_positive
    sum_positive = sum([r[i] for i in range(len(r)) if y[i]==1])
    auc = ((sum_positive - num_positive*(num_positive+1)/2.0) /
           (num_negative*num_positive))
    return auc

In [22]:
# validation

print('validating models...')

t0 = time()

all_y = []
all_pred = []

f_pred = [] 
f_pred = open('ftrl_pred.txt','w')
f_pred.write('y_actual,y_pred\n')


for e in range(epoch):
    loss = 0.
    count = 0
    for t, display_id, ad_id, x, y in data(train, D):  # data is a generator

        p = learner.predict(x)
        loss += logloss(p, y)
        learner.update(x, p, y)
        count+=1
        
        all_y.append(y)
        all_pred.append(p)
        f_pred.write('%s,%s\n' % (y, p))

        if count % 200000 == 0:
          print('processed %dth row' % count)
        if show_auc:
          pred_auc = auc(all_y, all_pred)
          
          
pred_auc = auc(all_y, all_pred)          
print('final auc: %.4f' % pred_auc)

f_pred.close()  

print('predict took %0.3fm' % ((time() - t0) / 60))
del all_y, all_pred

validating models...
processed 200000th row
processed 400000th row
processed 600000th row
processed 800000th row
processed 1000000th row
final auc: 0.7284
predict took 1.310m


In [23]:
with open(submission, 'w') as outfile:
    outfile.write('display_id,ad_id, prob\n')
    for t, display_id, ad_id, x, y in data(test, D):
        p = learner.predict(x)
        outfile.write('%s,%s,%s\n' % (display_id, ad_id, str(p)))

# map score

In [45]:
!pip3 install ml_metrics

  Running setup.py bdist_wheel for ml-metrics ... done
  Stored in directory: /content/.cache/pip/wheels/b3/61/2d/776be7b8a4f14c5db48c8e5451451cabc58dc6aa7ee3801163
Successfully built ml-metrics


In [46]:
import numpy as np 
import pandas as pd
from ml_metrics import mapk

In [69]:
# submission = df_test
df_test = pd.read_csv(submission)
df_test.head()

,display_id,ad_id,label_pred
0,15545104,1419,0.162134
1,15523904,784,0.196938
2,16731814,949,0.233614
3,13517470,492732,0.299224
4,14303795,492733,0.438912


In [70]:
#df_test.columns = ['display_id', 'ad_id', 'prob']

In [71]:
df_test.sort_values(['display_id', 'prob'], inplace=True, ascending=[True, False] )
df_test.head(20)

,display_id,ad_id,prob
3341,13352644,6186,0.389707
6492,13352644,497731,0.153318
233265,13352644,488003,0.090401
116694,13352644,504323,0.085584
118722,13352644,138832,0.085373
201397,13352644,98307,0.073456
26566,13352758,467799,0.297327
183379,13352758,39675,0.156494
230187,13352758,459432,0.129399
212799,13352758,285722,0.127339


In [74]:
raw_test = pd.read_csv(test)
raw_test = raw_test[['display_id','ad_id','label']]
raw_test.head()

/usr/local/envs/py3env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (25,30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,display_id,ad_id,label
0,15545104,1419,0
1,15523904,784,0
2,16731814,949,0
3,13517470,492732,1
4,14303795,492733,1


In [75]:
final = pd.merge(df_test, raw_test, on = ['display_id','ad_id'], how = 'left')
final.head(15)

,display_id,ad_id,prob,label
0,13352644,6186,0.389707,1
1,13352644,497731,0.153318,0
2,13352644,488003,0.090401,0
3,13352644,504323,0.085584,0
4,13352644,138832,0.085373,0
5,13352644,98307,0.073456,0
6,13352758,467799,0.297327,0
7,13352758,39675,0.156494,0
8,13352758,459432,0.129399,1
9,13352758,285722,0.127339,0


In [76]:
Y_ads = final[final.label == 1 ].ad_id.values.reshape(-1,1) # 실제 클릭된 ad_id
P_ads = final.groupby(by='display_id', sort=False).ad_id.apply( lambda x: x.values ).values  # 예측 순서대로 나열한 ad_id

In [77]:
print(Y_ads)
print(P_ads)

[[  6186]
 [459432]
 [ 81604]
 ...
 [492729]
 [175214]
 [179047]]
[array([  6186, 497731, 488003, 504323, 138832,  98307])
 array([467799,  39675, 459432, 285722])
 array([ 81604,  44953, 367363,  25876, 123813]) ...
 array([492729, 235450,  63884]) array([175214,  96199,  59607, 509329])
 array([536293, 293747, 179047, 365360])]


In [78]:
score = mapk( Y_ads, P_ads, 12 )
print("MAP: %.12f" % score)

MAP: 0.625901380764
